In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#.py files
import BCDR_utils as utils
import generate_data as generator
import automatic_patching as patching

In [ ]:
bcdrN = pd.read_csv('BCDR-DN01_dataset/bcdr_dn01_img.csv')
print('Normal Dataset: ',bcdrN.shape)
#bcdrN.head(10)

In [ ]:
bcdr1 = pd.read_csv('BCDR-D01_dataset/bcdr_d01_img.csv')
bcdr1_labels = pd.read_csv('BCDR-D01_dataset/bcdr_d01_features.csv')
print('Suspicious Dataset Nº 1: ',bcdr1.shape)
#bcdr1.head(10)

In [ ]:
normal_broken_paths= []
for path in bcdrN['image_filename']:
    normal_broken_paths.append('BCDR-DN01_dataset/'+path)
normal_paths_tif = utils.fix_path(normal_broken_paths)
normal_paths= generator.convert(normal_paths_tif)

In [ ]:
suspicious_broken_paths_1= []
for path in bcdr1['image_filename']:
    suspicious_broken_paths_1.append('BCDR-D01_dataset/'+path)
suspicious_paths_1_tif = utils.fix_path(suspicious_broken_paths_1)

suspicious_paths_1 = generator.convert(suspicious_paths_1_tif)

# Amostra do Algortimo de Patching

In [ ]:
patch_size = 200
background_percentage = 0.95

In [ ]:
mammogram_array = patching.raw_mammogram(suspicious_paths_1[4])
#plt.imshow(mammogram_array,cmap='gray')

In [ ]:
binarized_array = patching.binarize_breast_region(mammogram_array)
#plt.imshow(binarized_array,cmap='gray')

In [ ]:
random_patches_vertexes = patching.random_patch_corners(binarized_array,patch_size,background_percentage)
print('Random Patches:',len(random_patches_vertexes))

In [ ]:
sistematic_patches_vertexes = patching.sistematic_patch_corners(binarized_array,patch_size,background_percentage)
print('Sistematic Patches:',len(sistematic_patches_vertexes))

In [ ]:
cols = 5
rows = np.ceil(len(sistematic_patches_vertexes[30:60])/cols).astype(np.int_)
f,s = plt.subplots(rows,cols,figsize=(25,25))
i=0
for v in sistematic_patches_vertexes[60:90]:
    patch = mammogram_array[v[0]:v[1],v[2]:v[3]]
    s[i//cols,i%cols].imshow(patch,cmap='gray')
    i+=1

In [ ]:
mammogram_array = patching.raw_mammogram(suspicious_paths_1[4])
mammogram_with_random_centers = patching.show_centers(random_patches_vertexes,mammogram_array)
plt.imshow(mammogram_with_random_centers,cmap='gray')
#plt.imsave('random_centers.jpeg',mammogram_with_random_centers,cmap='gray')

In [ ]:
mammogram_array = patching.raw_mammogram(suspicious_paths_1[4])
mammogram_with_sistematic_centers = patching.show_centers(sistematic_patches_vertexes,mammogram_array)
plt.imshow(mammogram_with_sistematic_centers,cmap='gray')
#plt.imsave('sistematic_centers.jpeg',mammogram_with_sistematic_centers,cmap='gray')

## Saving Patches

In [ ]:
patch_size = 350
background_percentage = 0.95

In [ ]:
training_split = 0.7
n_suspicious_training = int(training_split*len(suspicious_paths_1))+1
n_normal_training = int(training_split*len(normal_paths))+1

In [ ]:
suspicious_training_path_list = suspicious_paths_1[:n_suspicious_training]
suspicious_validation_path_list = suspicious_paths_1[n_suspicious_training:]
normal_training_path_list = normal_paths[:n_normal_training]
normal_validation_path_list = normal_paths[n_normal_training:]

In [ ]:
patching_type = 'random'

In [ ]:
save_filename_folder = 'patches/R_350_patches/training_data/suspicious/'
patching.save_patches(suspicious_training_path_list,patch_size,background_percentage,save_filename_folder,patching_type)

In [ ]:
save_filename_folder = 'patches/R_350_patches/validation_data/suspicious/'
patching.save_patches(suspicious_validation_path_list,patch_size,background_percentage,save_filename_folder,patching_type)

In [ ]:
save_filename_folder = 'patches/R_350_patches/training_data/normal/'
patching.save_patches(normal_training_path_list,patch_size,background_percentage,save_filename_folder,patching_type)

In [ ]:
save_filename_folder = 'patches/R_350_patches/validation_data/normal/'
patching.save_patches(normal_validation_path_list,patch_size,background_percentage,save_filename_folder,patching_type)